In [1]:
GOOGLE_MAPS_API_KEY = "asdfsdfasdfasdfasdf-asdfasdf"

In [ ]:
# ───────────────────────────────────────────────────────────────
# MapFinderTool – Google-ADK functional tool
# Finds nearby doctors / hospitals via Google Maps Places API
# ───────────────────────────────────────────────────────────────
# pip install googlemaps
# Sub-agents call:
#   {"tool": "MapFinderTool",
#    "query": "cardiologist",
#    "location": {"lat": 26.1445, "lng": 91.7362},
#    "radius_m": 5000}
# The tool returns a JSON list of top results with distance, rating,
# address, and a Google-Maps link.
# ───────────────────────────────────────────────────────────────

from __future__ import annotations
import os, json
from typing import Dict, Any, List
from pydantic import BaseModel, Field
import googlemaps
from geopy.distance import geodesic  
#we'd be using this argument tyoe

# MapArgs:
#     query: str = Field(
#         description="Free-text search, e.g. 'cardiologist', 'hospital', 'nutritionist'."
#     )
#     location: Dict[str, float] = Field(
#         description="User location dict with 'lat' and 'lng' floats (WGS-84)."
#     )
#     radius_m: int = Field(
#         5000,
#         description="Search radius in metres (max 50 000 per Google Maps API).",
#     )
#     max_results: int = Field(
#         5, ge=1, le=20, description="Limit number of places returned."
#     )

ReturnType = List[Dict[str, Any]] 


def map_finder_tool(
    query: str,
    location: Optional[Dict[str, float]] = None,
    address_str: Optional[str] = None,
    radius_m: int = 5000,
    max_results: int = 5,
)-> ReturnType:
    """
    MapFinderTool looks at the query and based on it searches the location, in the specified radius.
    Args:
        1. query: str -> Free-text search, e.g. 'cardiologist', 'hospital', 'nutritionist'.
        2. location: Dict[str, float] ->User location dict with 'lat' and 'lng' floats (WGS-84).
        3. address_str: Optional[str] -> City + state (or any address) to geocode if lat/lng not given.
        4. radius_m: int -> Search radius in metres (max 50 000 per Google Maps API).
        5. max_results: int -> Limit number of places returned.
    Returns:
        a list of nearby places with name, address, rating, distance, url
    
    ."""

    gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
    # lat, lng = location["lat"], location["lng"]
    # ── Resolve coordinates ───────────────────────────────────
    if location:                                   # lat/lng already given
        lat, lng = location["lat"], location["lng"]
    else:                                          # need to geocode address_str
        geo = gmaps.geocode(address_str)
        if not geo:
            raise ValueError(f"Could not geocode '{address_str}'.")
        lat = geo[0]["geometry"]["location"]["lat"]
        lng = geo[0]["geometry"]["location"]["lng"]


    resp = gmaps.places(
        query=query,
        location=(lat, lng),
        radius=radius_m,
        type="doctor",             # bias toward medical facilities
    )
    places = resp.get("results", [])[: max_results]

    enriched: ReturnType = []
    for p in places:
        place_loc = p["geometry"]["location"]
        dist_km = round(
            geodesic((lat, lng), (place_loc["lat"], place_loc["lng"])).km, 2
        )
        enriched.append(
            {
                "name": p["name"],
                "address": p.get("formatted_address"),
                "rating": p.get("rating"),
                "distance_km": dist_km,
                "maps_url": f"https://www.google.com/maps/place/?q=place_id:{p['place_id']}",
            }
        )
    return enriched


In [20]:
results = map_finder_tool(
    query="Dentist",
    address_str="Uttarakhand, Dehradun",
    radius_m=8000,
    max_results=5,
)

for p in results:
    print(f"{p['name']} — {p['distance_km']} km — {p['rating']}★")
    print(" ", p['address'])
    print(" ", p['maps_url'])


Dr Mandoli Dental clinic — 6.61 km — 4.9★
  Buddha temple, Lane Near post office, road, Clement Town, Dehradun, Uttarakhand 248001
  https://www.google.com/maps/place/?q=place_id:ChIJEdVQT18rCTkRSyVA65FcQmE
Dr. Monika Koshwal — 2.69 km — 4.8★
  First Floor, victory point, Ballupur Rd, near PNB atm, Ram Vihar, Ballupur, Dehradun, Uttarakhand 248001
  https://www.google.com/maps/place/?q=place_id:ChIJb7j6kk8pCTkRengpSyAqd7Q
Dr Sawai's Dental Clinic — 2.64 km — 4.9★
  A-70, 534, 515, Kaulagarh Rd, Rajender Nagar, Dehradun, Uttarakhand 248001
  https://www.google.com/maps/place/?q=place_id:ChIJdWeHicMpCTkRN1CJ8Lhy17Q
Dr. Arora's Multispeciality Dental Clinic — 5.13 km — 4.9★
  First floor, 13/11 Canal road, Rajpur Rd, Dehradun, Uttarakhand 248001
  https://www.google.com/maps/place/?q=place_id:ChIJbXqeP4LXCDkRdjT0VK-tpKg
Dr Saurabh's Dental Clinic — 4.87 km — 4★
  Turner Rd, Morowala, Clement Town, Dehradun, Uttarakhand 248001
  https://www.google.com/maps/place/?q=place_id:ChIJu1FOzscrCTk